# Code for feature engineering of item, geo and UI

#### Author: </p>
Xu Chenqi  徐琛奇  1901212653   Quantitate Finance 19

#### Statement：</p>
1. The code part does not refer to the existing code about the problem. The code is all written by the author herself.</p>
2. Although this code is applicable to data for a period of 3 days, this code is versatile. By modifying the path and time variables, you can run the data for 2 days and 4 days.

In [1]:
from threading import Thread
import pandas as pd
from collections import Counter
import  numpy as np

In [6]:
loc = 'D:/Github/PHBS_MLF_2019/Data/Three Days/tianchi_fresh_comp_train_user_'
df1_1= pd.read_excel(loc + 'train1.xlsx')
df1_2= pd.read_excel(loc + 'train2.xlsx')
df1_3= pd.read_excel(loc + 'train3.xlsx')
df1_4= pd.read_excel(loc + 'train4.xlsx')
df1_5= pd.read_excel(loc + 'test.xlsx')
'''
train1_14,15,16,17
train2_13,14,15,16
train3_8,9,10,11
train4_7,8,9,10
test_15,16,17,18
'''

'\ntrain1_14,15,16,17\ntrain2_13,14,15,16\ntrain3_8,9,10,11\ntrain4_7,8,9,10\ntest_15,16,17,18\n'

In [9]:
for i in range(1,6):
    locals()['df1_'+str(i)]['time1']=locals()['df1_'+str(i)]['time'].map(lambda x:x.split(" ")[0])

df1=df1_1.loc[(df1_1["time1"]!="2014-12-17")]
df2=df1_2.loc[(df1_2["time1"]!="2014-12-16")]
df3=df1_3.loc[(df1_3["time1"]!="2014-12-11")]
df4=df1_4.loc[(df1_4["time1"]!="2014-12-10")]
df5=df1_5.loc[(df1_5["time1"]!="2014-12-18")]

for i in range(1,6):
    print(locals()['df'+str(i)].head())

    user_id    item_id  behavior_type user_geohash  item_category  \
0  10001082  150720867              1      95qoghe           3200   
1  10001082  220586551              1          NaN           7079   
2  10001082  209290607              1          NaN           5894   
3  10001082   22667958              1      95qogn4          10523   
4  10001082  125083630              1          NaN           4722   

            time       time1  
0  2014-12-15 08  2014-12-15  
1  2014-12-14 03  2014-12-14  
2  2014-12-14 16  2014-12-14  
3  2014-12-15 08  2014-12-15  
4  2014-12-14 03  2014-12-14  
    user_id    item_id  behavior_type user_geohash  item_category  \
0  10001082  150720867              1      95qoghe           3200   
1  10001082  275221686              1          NaN          10576   
2  10001082  220586551              1          NaN           7079   
3  10001082  209290607              1          NaN           5894   
4  10001082   22667958              1      95qogn4    

In [10]:
for i in range(1,6):
    locals()['df'+str(i)]=locals()['df'+str(i)].copy(deep=True)

def feature(df):
    df['behavior_type1']=df['behavior_type'].apply(lambda x: list(str(x)))
    df['user_id1'] = df['user_id'].apply(lambda x: [str(x)])
    df['item_id1'] = df['item_id'].apply(lambda x: [str(x)])
    df['user_item_pair']=df['user_id'].map(str)+'-'+df['item_id'].map(str)
    df['user_geohash1'] = df['user_geohash'].apply(lambda x: 0 if x=='nan' else str(x)[:3])
    #groupby
    a = df.groupby('user_id')['behavior_type1'].sum()
    b = df.groupby('item_id')['behavior_type1'].sum()
    c = df.groupby('item_id')['user_id1'].sum()
    d=df.groupby(['user_id','item_id'])['behavior_type1'].sum()
    e=df.groupby(['user_id','item_id'])['item_id1'].sum()
    f= df.groupby('user_geohash1')['behavior_type1'].sum()
    data = pd.DataFrame()
    data['Behavior_list'] = a
    data1 = pd.DataFrame()
    data1['Behavior_list'] = b
    data2 = pd.DataFrame()
    data2['user_id_list'] = c
    data3=pd.DataFrame()
    data3['Behavior_list']=d
    data4=pd.DataFrame()
    data4['item_id_list']=e 
    data5 = pd.DataFrame()
    data5['behavior_list'] = f
    # count the number of acts of each group
    data1['Number_of_acts'] = data2['user_id_list'].apply(lambda x: len(x))
    data3['Number_of_acts_UI'] = data4['item_id_list'].apply(lambda x: len(x))
    '''
    print('data1:',end='\n')
    print(data1.head(2),end='\n')
    print('data3:',end='\n')
    print(data3.head(2),end='\n')
    '''
############# Feature of item################
####### Statistics
    #Number of times the product was purchased
    data['2_item_buy'] = data['Behavior_list'].apply(lambda x: 0 if '4' not in x else Counter(x)['4'])
    #Number of users who purchased the item
    data1['2_item_buypeople'] = data1['Behavior_list'].apply(lambda x: 0 if '4' not in x else Counter(x)['4'])
    #Number of times the product was viewed
    data['2_item_view'] = data['Behavior_list'].apply(lambda x: 0 if '1' not in x else Counter(x)['1'])
    #Number of users who viewed the item
    data1['2_item_viewpeople'] = data1['Behavior_list'].apply(lambda x: 0 if '1' not in x else Counter(x)['1'])
    #Number of times the product was collected
    data['2_item_collect'] = data['Behavior_list'].apply(lambda x: 0 if '2' not in x else Counter(x)['2'])
    #Number of users who collected the item
    data1['2_item_collectpeople'] = data1['Behavior_list'].apply(lambda x: 0 if '2' not in x else Counter(x)['2'])
    #Number of times the product was carted
    data['2_item_add'] = data['Behavior_list'].apply(lambda x: 0 if '3' not in x else Counter(x)['3'])
    # Number of users who carted the item
    data1['2_item_addpeople'] = data1['Behavior_list'].apply(lambda x: 0 if '3' not in x else Counter(x)['3'])

####### Ratio
    #Number of times the product was purchased/Number of times the product was viewed
    data['2_item_buy_view'] = data['2_item_buy']/data['2_item_view']
    #Number of users who purchased the item/Number of users who viewed the item
    data1['2_item_buypeople_viewpeople'] = data1['2_item_buypeople']/data1['2_item_viewpeople']
    #Number of times the product was purchased/Number of times the product was collected
    data['2_item_buy_collect'] = data['2_item_buy'] / data['2_item_collect']
    #Number of users who purchased the item/Number of users who collected the item
    data1['2_item_buypeople_collectpeople'] = data1['2_item_buypeople']/data1['2_item_collectpeople']
    #Number of times the product was purchased/Number of times the product was carted
    data['2_item_buy_add'] = data['2_item_buy'] / data['2_item_add']
    #Number of users who purchased the item/Number of users who carted the item
    data1['2_item_buypeople_addpeople'] = data1['2_item_buypeople']/data1['2_item_addpeople']
    #Number of users who make multiple purchases/Number of users who purchased the product in the two days
    data1['2_item_frequentbuypeople_buypeople'] = data1['2_item_buypeople'] / data1['Number_of_acts']
    #Number of users who viewed the product multiple times/Number of users who purchased the product in the two days
    data1['2_item_frequentviewpeople_viewpeople'] = data1['2_item_viewpeople'] / data1['Number_of_acts']
    #Number of users who collected the product multiple times/Number of users who purchased the product in the two days 
    data1['2_item_frequentcollectpeople_collectpeople'] = data1['2_item_collectpeople'] / data1['Number_of_acts']
    #Number of users who carted the product multiple times/Number of users who purchased the product in the two days
    data1['2_item_frequentaddpeople_addpeople'] = data1['2_item_addpeople'] / data1['Number_of_acts']
    data = data.reset_index()
    data1 = data1.reset_index()
    '''
    print('data:',end='\n')
    print(data.head(2),end='\n')
    print('data1:',end='\n')
    print(data1.head(2),end='\n')
    print('item成功',end='\n')
    '''
############# Feature of GEO(geographic location)################
    data5['4_geo_view'] = data5['behavior_list'].apply(lambda x: 0 if '1' not in x else Counter(x)['1'])
    data5['4_geo_collect'] = data5['behavior_list'].apply(lambda x: 0 if '2' not in x else Counter(x)['2'])
    data5['4_geo_add'] = data5['behavior_list'].apply(lambda x: 0 if '3' not in x else Counter(x)['3'])
    data5['4_geo_buy'] = data5['behavior_list'].apply(lambda x: 0 if '4' not in x else Counter(x)['4'])
    data5['behavior_total'] = data5['behavior_list'].apply(lambda x: len(x))
    #Number of items purchased in the area / Total number of users in the area
    data5['4_geo_purchasepower'] = data5['4_geo_buy']/data5['behavior_total']
    #Number of items purchased in the area / Number of goods viewed in the area
    data5['4_geo_buy_view'] = data5['4_geo_buy']/data5['4_geo_view']
    #Number of goods purchased in the area / Number of goods collected in the area   
    data5['4_geo_buy_collect'] = data5['4_geo_buy']/data5['4_geo_collect']
    #Number of goods purchased in the area to / Number of goods carted in the area    
    data5['4_geo_buy_add'] = data5['4_geo_buy']/data5['4_geo_add']
    data5 = data5.reset_index()
    '''
    print('data5:',end='\n')
    print(data5.head(2),end='\n')
    print('GEO成功',end='\n')
    '''
############# Feature of UI################
    ####### Statistics
    #The number of times the user viewed the item
    data3['6_UI_useritemview'] = data3['Behavior_list'].apply(lambda x: 0 if '1' not in x else Counter(x)['1'])
    #The number of times the user collected the item
    data3['6_UI_useritemcollect'] = data3['Behavior_list'].apply(lambda x: 0 if '2' not in x else Counter(x)['2'])
    #The number of times the user carted the item
    data3['6_UI_useritemcart'] = data3['Behavior_list'].apply(lambda x: 0 if '3' not in x else Counter(x)['3'])
    #The number of times the user purchased the item
    data3['6_UI_useritembuy'] = data3['Behavior_list'].apply(lambda x: 0 if '4' not in x else Counter(x)['4'])
    ####### Ratio
    #The number of times the user views the item / The user's total views of all items
    data3['6_UI_useritemview_usertotalview'] = data3['6_UI_useritemview'] / data3['Number_of_acts_UI']
    #The number of times the user collects the item / The user's total collection of all items
    data3['6_UI_useritemcollect_usertotalcollect'] = data3['6_UI_useritemcollect'] / data3['Number_of_acts_UI']
    #The number of times the user carts the item / The user's total amount of all items added to the shopping cart
    data3['6_UI_useritemcart_usertotalcart'] = data3['6_UI_useritemcart'] / data3['Number_of_acts_UI']
    #The number of times the user purchases the item / The user's total purchases of all items
    data3['6_UI_useritembuy_usertotalbuy'] = data3['6_UI_useritembuy'] / data3['Number_of_acts_UI']
    data3 = data3.reset_index()
    '''
    print('data3:',end='\n')
    print(data3.head(2),end='\n')
    print('UI成功',end='\n')
    '''
#############Merge column#############
    df = pd.merge(df, data, how='left', on=['user_id'])
    df = pd.merge(df, data1, how='left', on=['item_id'])
    df = pd.merge(df, data5, how='left', on=['user_geohash1'])
    r_df= pd.merge(df, data3, how='left', on=['user_id', 'item_id'])
    print('success!!',end='\n')
    return r_df

In [11]:
for i in range(1,6):
    locals()['rr_df'+str(i)]=feature(locals()['df'+str(i)]) 
    print(locals()['rr_df'+str(i)].head(2),end='\n')
    locals()['col_df'+str(i)]= [x for x in locals()['rr_df'+str(i)].columns]
    print(locals()['col_df'+str(i)])

success!!
    user_id    item_id  behavior_type user_geohash  item_category  \
0  10001082  150720867              1      95qoghe           3200   
1  10001082  220586551              1          NaN           7079   

            time       time1 behavior_type1    user_id1     item_id1  ...  \
0  2014-12-15 08  2014-12-15            [1]  [10001082]  [150720867]  ...   
1  2014-12-14 03  2014-12-14            [1]  [10001082]  [220586551]  ...   

  Behavior_list Number_of_acts_UI 6_UI_useritemview  6_UI_useritemcollect  \
0           [1]                 1                 1                     0   
1  [1, 4, 1, 1]                 4                 3                     0   

   6_UI_useritemcart  6_UI_useritembuy  6_UI_useritemview_usertotalview  \
0                  0                 0                             1.00   
1                  0                 1                             0.75   

   6_UI_useritemcollect_usertotalcollect  6_UI_useritemcart_usertotalcart  \
0              

success!!
    user_id    item_id  behavior_type user_geohash  item_category  \
0  10001082  150720867              1      95qoghe           3200   
1  10001082   22667958              1      95qogn4          10523   

            time       time1 behavior_type1    user_id1     item_id1  ...  \
0  2014-12-15 08  2014-12-15            [1]  [10001082]  [150720867]  ...   
1  2014-12-15 08  2014-12-15            [1]  [10001082]   [22667958]  ...   

  Behavior_list Number_of_acts_UI 6_UI_useritemview  6_UI_useritemcollect  \
0           [1]                 1                 1                     0   
1     [1, 1, 1]                 3                 3                     0   

   6_UI_useritemcart  6_UI_useritembuy  6_UI_useritemview_usertotalview  \
0                  0                 0                              1.0   
1                  0                 0                              1.0   

   6_UI_useritemcollect_usertotalcollect  6_UI_useritemcart_usertotalcart  \
0              

In [30]:
####Here you need to manually set the file path
loc = 'D:/Github/PHBS_MLF_2019/Data/Three Days/user_item_pair_'
train5= pd.read_excel(loc + 'test_form.xlsx')
train1234_1= pd.read_excel(loc + 'train_form.xlsx')
train5
train1234_1
'''
train5-18
train1234-10,11,16,17
'''
'''
train1_14,15,16,17
train2_13,14,15,16
train3_8,9,10,11
train4_7,8,9,10
test_15,16,17,18
'''

'\ntrain1_14,15,16,17\ntrain2_13,14,15,16\ntrain3_8,9,10,11\ntrain4_7,8,9,10\ntest_15,16,17,18\n'

In [33]:
train1234=train1234_1.reset_index()
train1234['id']=train1234['index']+1
train1234=train1234.drop(['index'],axis=1)
train1234.head(5)

,id,user_id,item_id,label,time
0,1,10001082,150720867,0,2014-12-17
1,2,10001082,220586551,0,2014-12-17
2,3,10001082,209290607,0,2014-12-17
3,4,10001082,22667958,0,2014-12-17
4,5,10001082,125083630,0,2014-12-17


In [35]:
#####The date needs to be set manually here
train1=train1234.loc[(train1234["time"]=="2014-12-17")]
train2=train1234.loc[(train1234["time"]=="2014-12-16")]
train3=train1234.loc[(train1234["time"]=="2014-12-11")]
train4=train1234.loc[(train1234["time"]=="2014-12-10")]
for i in range(1,5):
    print(locals()['train'+str(i)].head(2))

   id   user_id    item_id  label        time
0   1  10001082  150720867      0  2014-12-17
1   2  10001082  220586551      0  2014-12-17
    id   user_id    item_id  label        time
30  31  10001082  150720867      0  2014-12-16
31  32  10001082  275221686      0  2014-12-16
    id   user_id    item_id  label        time
79  80  10001082  285259775      0  2014-12-11
80  81  10001082  275221686      0  2014-12-11
      id   user_id    item_id  label        time
111  112  10001082  285259775      0  2014-12-10
112  113  10001082  275221686      0  2014-12-10


In [36]:
for i in range(1,6):
    locals()['df'+str(i)]=locals()['df'+str(i)].copy(deep=True)

#######match the columns of train/test with df
drop = ['user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category',
            'time','time1', 'behavior_type1', 'user_id 1', 'user_id1','item_id1', 
            'user_item_pair', 'user_geohash1', 'Behavior_list_x','Behavior_list_y', 
            'Number_of_acts','behavior_total','behavior_list','Behavior_list', 'Number_of_acts_UI']

for i in range(1,6):
    locals()['train_col_'+str(i)]= [x for x in locals()['rr_df'+str(i)].columns if x not in drop]
    locals()['train'+str(i)][locals()['train_col_'+str(i)]] = locals()['rr_df'+str(i)][locals()['train_col_'+str(i)]]

In [37]:
train1_col= [x for x in train1.columns]
print(train1_col)

['id', 'user_id', 'item_id', 'label', 'time', '2_item_buy', '2_item_view', '2_item_collect', '2_item_add', '2_item_buy_view', '2_item_buy_collect', '2_item_buy_add', '2_item_buypeople', '2_item_viewpeople', '2_item_collectpeople', '2_item_addpeople', '2_item_buypeople_viewpeople', '2_item_buypeople_collectpeople', '2_item_buypeople_addpeople', '2_item_frequentbuypeople_buypeople', '2_item_frequentviewpeople_viewpeople', '2_item_frequentcollectpeople_collectpeople', '2_item_frequentaddpeople_addpeople', '4_geo_view', '4_geo_collect', '4_geo_add', '4_geo_buy', '4_geo_purchasepower', '4_geo_buy_view', '4_geo_buy_collect', '4_geo_buy_add', '6_UI_useritemview', '6_UI_useritemcollect', '6_UI_useritemcart', '6_UI_useritembuy', '6_UI_useritemview_usertotalview', '6_UI_useritemcollect_usertotalcollect', '6_UI_useritemcart_usertotalcart', '6_UI_useritembuy_usertotalbuy']


In [44]:
train_1=train1.append([train2,train3,train4])
train_1=train_1.fillna(0)
train_1=train_1.replace(float('inf'),0)
train_2=train_1.sort_index(axis = 0,ascending = True,by = 'id')
train_2.head(10)

train5=train5.fillna(0)
train5=train5.replace(float('inf'),0)
test=train5
test

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  after removing the cwd from sys.path.


,user_id,item_id,label,time,2_item_buy,2_item_view,2_item_collect,2_item_add,2_item_buy_view,2_item_buy_collect,...,4_geo_buy_collect,4_geo_buy_add,6_UI_useritemview,6_UI_useritemcollect,6_UI_useritemcart,6_UI_useritembuy,6_UI_useritemview_usertotalview,6_UI_useritemcollect_usertotalcollect,6_UI_useritemcart_usertotalcart,6_UI_useritembuy_usertotalbuy
0,10001082,150720867,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,1,0,0,0,1.000000,0.000000,0.000000,0.00
1,10001082,22667958,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,3,0,0,0,1.000000,0.000000,0.000000,0.00
2,10001082,133020432,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,1,0,0,0,1.000000,0.000000,0.000000,0.00
3,10001082,275221686,0,2014-12-18,0,13,0,0,0.000000,0.0,...,0.47806,0.357102,4,0,0,0,1.000000,0.000000,0.000000,0.00
4,10001082,582649,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,1,0,0,0,1.000000,0.000000,0.000000,0.00
5,10001082,377161387,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,2,0,0,0,1.000000,0.000000,0.000000,0.00
6,10001082,275221686,0,2014-12-18,0,13,0,0,0.000000,0.0,...,0.47806,0.357102,4,0,0,0,1.000000,0.000000,0.000000,0.00
7,10001082,275221686,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,4,0,0,0,1.000000,0.000000,0.000000,0.00
8,10001082,275221686,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,4,0,0,0,1.000000,0.000000,0.000000,0.00
9,10001082,22667958,0,2014-12-18,0,13,0,0,0.000000,0.0,...,21.00000,0.940299,3,0,0,0,1.000000,0.000000,0.000000,0.00


In [45]:
train_2

,id,user_id,item_id,label,time,2_item_buy,2_item_view,2_item_collect,2_item_add,2_item_buy_view,...,4_geo_buy_collect,4_geo_buy_add,6_UI_useritemview,6_UI_useritemcollect,6_UI_useritemcart,6_UI_useritembuy,6_UI_useritemview_usertotalview,6_UI_useritemcollect_usertotalcollect,6_UI_useritemcart_usertotalcart,6_UI_useritembuy_usertotalbuy
0,1,10001082,150720867,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.000000,0.393204,1.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
1,2,10001082,220586551,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,3.0,0.0,0.0,1.0,0.75,0.0,0.0,0.25
2,3,10001082,209290607,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,1.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
3,4,10001082,22667958,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.000000,0.393204,3.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
4,5,10001082,125083630,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,1.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
5,6,10001082,133020432,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.000000,0.393204,1.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
6,7,10001082,377161387,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.000000,0.393204,2.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
7,8,10001082,110790001,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,1.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00
8,9,10001082,220586551,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,3.0,0.0,0.0,1.0,0.75,0.0,0.0,0.25
9,10,10001082,85851739,0,2014-12-17,1.0,29.0,0.0,0.0,0.034483,...,0.491037,0.356335,4.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00


In [46]:
train_2.to_excel('D:/Github/PHBS_MLF_2019/Data/Four Days/result/user_item_pair_train_XCQ.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Github/PHBS_MLF_2019/Data/Four Days/result/user_item_pair_train_XCQ.xlsx'

In [ ]:
test.to_excel('D:/Github/PHBS_MLF_2019/Data/Four Days/result/user_item_pair_test_XCQ.xlsx')